In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt

from torch_geometric.datasets import ModelNet
import torch_geometric.transforms as T

import mlflow
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch_geometric.nn import DynamicEdgeConv, global_max_pool

from pointnet import PointNetCls
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score

In [2]:
import pptk

def view(points):
    v = pptk.viewer(points)
    v.attributes(points)
    v.set(point_size=0.01)
    # selected = points[v.get('selected')]
    return v

def plot_conf_matrix(conf_mtrx, labels, file_path='temp_conf_mtrx.png'):
    df_cm = pd.DataFrame(np.array(conf_mtrx), index=labels, columns=labels)
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True)
    plt.savefig(file_path)
    plt.close()
    
def get_path_of_last_model():
    models_path = 'models'
    files = list(filter(lambda f: os.path.isfile(os.path.join(models_path, f)) and f.endswith('.pth'), os.listdir(models_path) ))
    if len(files) == 0:
        return None, 0
    files.sort(key=lambda f: int(f.split('.')[0].split('_')[-1] ))
    return os.path.join(models_path, files[-1]), int(files[-1].split('.')[0].split('_')[-1])

def test_model_full(classifier, test_data, num2cat, step=0, model_epoch_cumulatiove_base=0):
    all_labels = []
    all_choice = []
    for j, data in enumerate(test_loader, 0):
        points, labels = data
        points = points.transpose(2, 1)
        points, labels = points.to(device), labels.to(device)
        classifier = classifier.eval()
        with torch.no_grad():
            pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        loss = F.nll_loss(pred, labels)
        pred_choice = pred.data.max(1)[1]
            
        all_labels.append(labels.cpu().numpy())
        all_choice.append(pred_choice.cpu().numpy())
            
    all_labels = np.concatenate(all_labels)
    all_choice = np.concatenate(all_choice)
    test_acc = accuracy_score(all_labels, all_choice)
    print(blue('epoch %d: %d/%d | test loss: %f | test acc: %f') % (model_epoch_cumulatiove_base+epoch+1, i+1, num_batch+1, loss.item(), test_acc))

    cnf_mtrx = confusion_matrix(all_labels, all_choice, labels=sorted(list(num2cat)))
    conf_mtrx_file_path = os.path.join("temp", f"test_cnf_mtrx_{epoch}_{i}.png")
    plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
    mlflow.log_artifact(conf_mtrx_file_path)
    mlflow.log_metric('test_acc', test_acc, step=step)
    return test_acc
    
def test_model_simple(model, test_loader, step=0):
    j, data = next(enumerate(test_loader, 0))
    points, labels = data
    points = points.transpose(2, 1)
    points, labels = points.to(device), labels.to(device)
    classifier = classifier.eval()
    with torch.no_grad():
        pred, _ = classifier(points)
    pred = pred.view(-1, num_classes)
    loss = F.nll_loss(pred, labels)
    pred_choice = pred.data.max(1)[1]
    correct = pred_choice.eq(labels.data).cpu().sum()
    test_acc = correct.item() / float(batchsize)
    print(blue('epoch %d: %d/%d | test loss: %f | test acc: %f') % (model_epoch_cumulatiove_base+epoch+1, i+1, num_batch+1, loss.item(), test_acc))

    # log test
    cnf_mtrx = confusion_matrix(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(list(num2cat)))
    conf_mtrx_file_path = os.path.join("temp", f"test_cnf_mtrx_{epoch}_{i}.png")
    plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
    mlflow.log_artifact(conf_mtrx_file_path)
    mlflow.log_metric('test_acc', np.mean(test_acc_epoch), step=step)
    return test_acc

In [3]:
batchsize = 32
blue = lambda x: '\033[94m' + x + '\033[0m'
yellow = lambda x: '\033[93m' + x + '\033[0m'
red = lambda x: '\033[91m' + x + '\033[0m'
pre_transform, transform = T.NormalizeScale(), T.SamplePoints(1024)
train_dataset = ModelNet('../data/modelnet10', '10', True, transform, pre_transform)
test_dataset = ModelNet('../data/modelnet10', '10', False, transform, pre_transform)

class Adapter:
    def __init__(self, pg_dataset):
        self.pg_dataset = pg_dataset

    def __len__(self):
        return len(self.pg_dataset)
    
    def __getitem__(self, idx):
        return self.pg_dataset[idx].pos.numpy(), self.pg_dataset[idx].y.numpy().item() 
    
print("train size: ", len(train_dataset), len(train_dataset)//batchsize)
print("test size: ", len(test_dataset), len(test_dataset)//batchsize)
train_loader = torch.utils.data.DataLoader(Adapter(train_dataset), batch_size=batchsize, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(Adapter(test_dataset), batch_size=batchsize, shuffle=False, num_workers=0)
num2cat = dict(zip(range(10), train_dataset.raw_file_names))

train size:  3991 124
test size:  908 28


In [6]:
num_classes = len(num2cat)
num_batch = len(train_dataset)/batchsize

classifier = PointNetCls(k=num_classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier.to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.001)    
scheduler = StepLR(optimizer, step_size=1, gamma=0.99)

model_path, model_epoch_cumulatiove_base = get_path_of_last_model()
model_epoch_cumulatiove_base += 1
if model_path:
    print('Loading model from: {}'.format(model_path))
    mlflow.log_param('start', model_path)
    classifier.load_state_dict(torch.load(model_path))
else:
    print('Start training from zero!')
    mlflow.log_param('start', 'From zero')

mlflow.log_param('start epoch', model_epoch_cumulatiove_base)
step = 0

Loading model from: models\model_10_model_9.pth


In [7]:
for epoch in range(10):
    train_acc_epoch, test_acc_epoch = [], []
    for i, data in enumerate(train_loader):
        points, labels = data
        points = points.transpose(2, 1)
        points, labels = points.to(device), labels.to(device)
        optimizer.zero_grad()
        classifier = classifier.train()
        pred, _ = classifier(points)
        pred = pred.view(-1, num_classes)
        loss = F.nll_loss(pred, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        pred_choice = pred.data.max(1)[1]
        correct = pred_choice.eq(labels.data).cpu().sum()
        train_acc = correct.item() / float(batchsize)
        print('epoch %d: %d/%d | train loss: %f | train acc: %f' % (model_epoch_cumulatiove_base+epoch+1, i+1, num_batch+1, loss.item(), train_acc))
        train_acc_epoch.append(train_acc)
        
        # log train
        encountered_class_nums = np.unique( np.array(labels.cpu().tolist()))
        prec, recall, f1, _ = precision_recall_fscore_support(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(encountered_class_nums))
        for j, num in enumerate(encountered_class_nums):
            name = num2cat[num]
            mlflow.log_metrics({
                f"{name}_acc": prec[j],
                f"{name}_recall": recall[j],
                f"{name}_f1": f1[j],
            }, step=step)
        cnf_mtrx = confusion_matrix(labels.cpu().tolist(), pred_choice.cpu().tolist(), labels=sorted(list(num2cat)))
        conf_mtrx_file_path = os.path.join("temp", f"train_cnf_mtrx_{epoch}_{i}.png")
        plot_conf_matrix(cnf_mtrx, [num2cat[num] for num in sorted(list(num2cat))], conf_mtrx_file_path)
        mlflow.log_artifact(conf_mtrx_file_path)
        mlflow.log_metric('train_acc', train_acc, step=step)

        if (i+1) % 10 == 0:
#             test_acc_epoch.append(test_model_simple(classifier, test_loader, step))
            test_acc_epoch.append(test_model_full(classifier, test_dataset, num2cat, step, model_epoch_cumulatiove_base))
            
        # to next batch
        step += 1
    
    # to next epoch
    print(yellow('epoch %d | mean train acc: %f') % (model_epoch_cumulatiove_base+epoch+1, np.mean(train_acc_epoch)))
    print(red('epoch %d | mean test acc: %f') % (model_epoch_cumulatiove_base+epoch+1, np.mean(test_acc_epoch)))
    torch.save(classifier.state_dict(), '%s/%s_model_%d.pth' % ('models', 'model_10', epoch))

epoch 11: 1/125 | train loss: 0.275583 | train acc: 0.875000
epoch 11: 2/125 | train loss: 0.439769 | train acc: 0.781250
epoch 11: 3/125 | train loss: 0.345226 | train acc: 0.875000
epoch 11: 4/125 | train loss: 0.118115 | train acc: 0.968750
epoch 11: 5/125 | train loss: 0.686675 | train acc: 0.718750
epoch 11: 6/125 | train loss: 0.242373 | train acc: 0.906250
epoch 11: 7/125 | train loss: 0.432911 | train acc: 0.843750
epoch 11: 8/125 | train loss: 0.722645 | train acc: 0.718750
epoch 11: 9/125 | train loss: 0.386642 | train acc: 0.843750
epoch 11: 10/125 | train loss: 0.317425 | train acc: 0.937500
epoch 11: 10/125 | test loss: 0.122314 | test acc: 0.806167
epoch 11: 11/125 | train loss: 0.516232 | train acc: 0.750000
epoch 11: 12/125 | train loss: 0.597246 | train acc: 0.843750
epoch 11: 13/125 | train loss: 0.390845 | train acc: 0.843750
epoch 11: 14/125 | train loss: 0.227128 | train acc: 0.906250
epoch 11: 15/125 | train loss: 0.135304 | train acc: 1.000000
epoch 11: 16/125 | 

epoch 11: 121/125 | train loss: 0.209844 | train acc: 0.937500
epoch 11: 122/125 | train loss: 0.255058 | train acc: 0.843750
epoch 11: 123/125 | train loss: 0.373178 | train acc: 0.875000
epoch 11: 124/125 | train loss: 0.358593 | train acc: 0.843750
epoch 11: 125/125 | train loss: 0.223243 | train acc: 0.625000
epoch 11 | mean train acc: 0.876250
epoch 11 | mean test acc: 0.825808
epoch 12: 1/125 | train loss: 0.226599 | train acc: 0.937500
epoch 12: 2/125 | train loss: 0.355719 | train acc: 0.875000
epoch 12: 3/125 | train loss: 0.135115 | train acc: 0.968750
epoch 12: 4/125 | train loss: 0.226976 | train acc: 0.937500
epoch 12: 5/125 | train loss: 0.162746 | train acc: 0.968750
epoch 12: 6/125 | train loss: 0.300707 | train acc: 0.906250
epoch 12: 7/125 | train loss: 0.400740 | train acc: 0.906250
epoch 12: 8/125 | train loss: 0.198019 | train acc: 0.906250
epoch 12: 9/125 | train loss: 0.218673 | train acc: 0.875000
epoch 12: 10/125 | train loss: 0.248391 | train acc: 0.906250
epo

epoch 12: 116/125 | train loss: 0.167663 | train acc: 0.906250
epoch 12: 117/125 | train loss: 0.435545 | train acc: 0.843750
epoch 12: 118/125 | train loss: 0.426009 | train acc: 0.906250
epoch 12: 119/125 | train loss: 0.346382 | train acc: 0.906250
epoch 12: 120/125 | train loss: 0.077924 | train acc: 1.000000
epoch 12: 120/125 | test loss: 0.036251 | test acc: 0.872247
epoch 12: 121/125 | train loss: 0.373868 | train acc: 0.937500
epoch 12: 122/125 | train loss: 0.195024 | train acc: 0.906250
epoch 12: 123/125 | train loss: 0.114135 | train acc: 0.968750
epoch 12: 124/125 | train loss: 0.229346 | train acc: 0.906250
epoch 12: 125/125 | train loss: 0.492783 | train acc: 0.625000
epoch 12 | mean train acc: 0.908750
epoch 12 | mean test acc: 0.859123
epoch 13: 1/125 | train loss: 0.308717 | train acc: 0.843750
epoch 13: 2/125 | train loss: 0.417101 | train acc: 0.781250
epoch 13: 3/125 | train loss: 0.324426 | train acc: 0.906250
epoch 13: 4/125 | train loss: 0.239396 | train acc: 0.9

epoch 13: 110/125 | test loss: 0.008598 | test acc: 0.873348
epoch 13: 111/125 | train loss: 0.137923 | train acc: 0.968750
epoch 13: 112/125 | train loss: 0.083145 | train acc: 1.000000
epoch 13: 113/125 | train loss: 0.321972 | train acc: 0.906250
epoch 13: 114/125 | train loss: 0.306371 | train acc: 0.906250
epoch 13: 115/125 | train loss: 0.242096 | train acc: 0.843750
epoch 13: 116/125 | train loss: 0.205895 | train acc: 0.968750
epoch 13: 117/125 | train loss: 0.219234 | train acc: 0.906250
epoch 13: 118/125 | train loss: 0.176260 | train acc: 0.906250
epoch 13: 119/125 | train loss: 0.144338 | train acc: 0.937500
epoch 13: 120/125 | train loss: 0.181100 | train acc: 0.937500
epoch 13: 120/125 | test loss: 0.016403 | test acc: 0.872247
epoch 13: 121/125 | train loss: 0.094931 | train acc: 0.968750
epoch 13: 122/125 | train loss: 0.220244 | train acc: 0.937500
epoch 13: 123/125 | train loss: 0.126453 | train acc: 0.968750
epoch 13: 124/125 | train loss: 0.110109 | train acc: 0.937

epoch 14: 105/125 | train loss: 0.285196 | train acc: 0.875000
epoch 14: 106/125 | train loss: 0.168108 | train acc: 0.906250
epoch 14: 107/125 | train loss: 0.462765 | train acc: 0.875000
epoch 14: 108/125 | train loss: 0.124348 | train acc: 0.968750
epoch 14: 109/125 | train loss: 0.108646 | train acc: 0.968750
epoch 14: 110/125 | train loss: 0.254882 | train acc: 0.875000
epoch 14: 110/125 | test loss: 0.008456 | test acc: 0.867841
epoch 14: 111/125 | train loss: 0.149001 | train acc: 0.937500
epoch 14: 112/125 | train loss: 0.266612 | train acc: 0.937500
epoch 14: 113/125 | train loss: 0.141330 | train acc: 0.937500
epoch 14: 114/125 | train loss: 0.188786 | train acc: 0.968750
epoch 14: 115/125 | train loss: 0.086859 | train acc: 0.968750
epoch 14: 116/125 | train loss: 0.322495 | train acc: 0.906250
epoch 14: 117/125 | train loss: 0.357744 | train acc: 0.937500
epoch 14: 118/125 | train loss: 0.194554 | train acc: 0.906250
epoch 14: 119/125 | train loss: 0.143848 | train acc: 0.9

epoch 15: 100/125 | train loss: 0.253271 | train acc: 0.906250
epoch 15: 100/125 | test loss: 0.009195 | test acc: 0.863436
epoch 15: 101/125 | train loss: 0.288831 | train acc: 0.906250
epoch 15: 102/125 | train loss: 0.109381 | train acc: 0.937500
epoch 15: 103/125 | train loss: 0.180588 | train acc: 0.968750
epoch 15: 104/125 | train loss: 0.150044 | train acc: 1.000000
epoch 15: 105/125 | train loss: 0.375429 | train acc: 0.812500
epoch 15: 106/125 | train loss: 0.287734 | train acc: 0.906250
epoch 15: 107/125 | train loss: 0.204048 | train acc: 0.937500
epoch 15: 108/125 | train loss: 0.613562 | train acc: 0.812500
epoch 15: 109/125 | train loss: 0.361727 | train acc: 0.812500
epoch 15: 110/125 | train loss: 0.182752 | train acc: 0.937500
epoch 15: 110/125 | test loss: 0.006696 | test acc: 0.867841
epoch 15: 111/125 | train loss: 0.254331 | train acc: 0.875000
epoch 15: 112/125 | train loss: 0.239904 | train acc: 0.937500
epoch 15: 113/125 | train loss: 0.152545 | train acc: 0.937

epoch 16: 94/125 | train loss: 0.243278 | train acc: 0.968750
epoch 16: 95/125 | train loss: 0.224547 | train acc: 0.875000
epoch 16: 96/125 | train loss: 0.099645 | train acc: 0.968750
epoch 16: 97/125 | train loss: 0.090756 | train acc: 0.968750
epoch 16: 98/125 | train loss: 0.074739 | train acc: 1.000000
epoch 16: 99/125 | train loss: 0.141339 | train acc: 1.000000
epoch 16: 100/125 | train loss: 0.265886 | train acc: 0.906250
epoch 16: 100/125 | test loss: 0.007120 | test acc: 0.865639
epoch 16: 101/125 | train loss: 0.178677 | train acc: 0.937500
epoch 16: 102/125 | train loss: 0.220614 | train acc: 0.875000
epoch 16: 103/125 | train loss: 0.104607 | train acc: 0.968750
epoch 16: 104/125 | train loss: 0.485740 | train acc: 0.812500
epoch 16: 105/125 | train loss: 0.281542 | train acc: 0.875000
epoch 16: 106/125 | train loss: 0.100592 | train acc: 0.968750
epoch 16: 107/125 | train loss: 0.319211 | train acc: 0.906250
epoch 16: 108/125 | train loss: 0.343199 | train acc: 0.906250


epoch 17: 89/125 | train loss: 0.311685 | train acc: 0.968750
epoch 17: 90/125 | train loss: 0.107478 | train acc: 0.937500
epoch 17: 90/125 | test loss: 0.005632 | test acc: 0.870044
epoch 17: 91/125 | train loss: 0.144829 | train acc: 0.937500
epoch 17: 92/125 | train loss: 0.369739 | train acc: 0.906250
epoch 17: 93/125 | train loss: 0.287053 | train acc: 0.906250
epoch 17: 94/125 | train loss: 0.261411 | train acc: 0.906250
epoch 17: 95/125 | train loss: 0.312565 | train acc: 0.812500
epoch 17: 96/125 | train loss: 0.425787 | train acc: 0.906250
epoch 17: 97/125 | train loss: 0.275452 | train acc: 0.906250
epoch 17: 98/125 | train loss: 0.136463 | train acc: 0.937500
epoch 17: 99/125 | train loss: 0.099547 | train acc: 1.000000
epoch 17: 100/125 | train loss: 0.099321 | train acc: 1.000000
epoch 17: 100/125 | test loss: 0.010460 | test acc: 0.870044
epoch 17: 101/125 | train loss: 0.277866 | train acc: 0.875000
epoch 17: 102/125 | train loss: 0.290452 | train acc: 0.906250
epoch 17

KeyboardInterrupt: 

In [55]:
num2class = dict(zip(range(10), train_loader.dataset.raw_file_names))

In [68]:
pointss = data.pos.numpy()
for btch in np.unique(data.batch.numpy()):
    points = pointss[data.batch == btch]
    print(num2class[data.y[btch].numpy().item()])
    print(np.min(points[:, 0], axis=0), np.max(points[:, 0], axis=0), np.max(points[:, 0], axis=0) - np.min(points[:, 0], axis=0))
    print(np.min(points[:, 1], axis=0), np.max(points[:, 1], axis=0), np.max(points[:, 1], axis=0) - np.min(points[:, 1], axis=0))
    print(np.min(points[:, 2], axis=0), np.max(points[:, 2], axis=0), np.max(points[:, 2], axis=0) - np.min(points[:, 2], axis=0))
    print()

bed
-0.8752633 0.89645064 1.771714
-0.99999905 0.89762324 1.8976223
-0.231328 0.4611699 0.6924979

desk
-0.25317165 0.8512118 1.1043835
-0.95747113 0.999999 1.9574702
-0.6959119 0.12586308 0.82177496

sofa
-0.71917844 0.6585192 1.3776977
-0.8147365 0.99999905 1.8147355
-0.1762224 0.4483075 0.6245299

chair
-0.6121519 0.58003956 1.1921915
-0.8185277 0.99923277 1.8177605
-0.56900716 0.8437649 1.412772

table
-0.2510807 0.5558331 0.8069138
-0.36220503 0.36191383 0.7241188
-0.999999 0.10629373 1.1062927

night_stand
-0.10616867 0.7600212 0.8661899
-0.79955983 0.79955995 1.5991198
-0.999999 0.9989007 1.9988997

monitor
-0.3327983 0.999999 1.3327973
-0.66639864 0.66639864 1.3327973
-0.5120062 0.8207911 1.3327973

monitor
-0.1636657 0.39862558 0.56229126
-0.6964982 0.67877895 1.3752772
-0.28227845 0.999999 1.2822775

sofa
-0.92549175 0.36221954 1.2877113
-0.46057156 0.9548646 1.4154361
-0.2584274 0.10073888 0.3591663

sofa
-0.5206773 0.2951557 0.81583303
-0.99939686 0.99364084 1.9930377
-0.48

In [71]:
for data in train_loader:
    pass
    break
    
print(data.y.numpy())
points = data.pos.numpy()[data.batch == 0]
view(points)

[1 7 1 7 7 8 1 7 2 1 6 7 2 4 9 2 5 9 6 8 7 2 8 9 1 5 9 3 2 7 5 7]
